# Meta_Features

In [38]:
import openml
from amltk.metalearning import compute_metafeatures
import pandas as pd

Los metafeatures que aparecen en openml.

In [39]:

# descarga un dataset de OpenML con ID 31.

dataset = openml.datasets.get_dataset(
    31,  # id del dataset
    download_data=True, # descarga los datos del dataset
    download_features_meta_data=False, # no descarga información extra sobre las características.
    download_qualities=False, # no descarga estadísticas de calidad del dataset
)

# obtiene los datos reales del dataset.
X, y, _, _ = dataset.get_data(
    dataset_format="dataframe", # convierte los datos en un dataframe pandas
    target=dataset.default_target_attribute, # selecciona la columna objetivo del dataset
)

# funcion que calcula metafeatures para un dataset.
mfs = compute_metafeatures(X, y)

print("----------------------------------------------------")
print(f"La cantidad de metafeures calculadas es: {len(mfs)}")
print("----------------------------------------------------")
mfs

Could not download file from https://openml.org/datasets/0000/0031/dataset_31.pq: HTTPSConnectionPool(host='www.openml.org', port=443): Max retries exceeded with url: https://www.openml.org/datasets/0000/0031/dataset_31.pq (Caused by ResponseError('too many redirects'))
Failed to download parquet, fallback on ARFF.


----------------------------------------------------
La cantidad de metafeures calculadas es: 30
----------------------------------------------------


instance_count                                           1000.000000
log_instance_count                                          6.907755
number_of_classes                                           2.000000
number_of_features                                         20.000000
log_number_of_features                                      2.995732
percentage_missing_values                                   0.000000
percentage_of_instances_with_missing_values                 0.000000
percentage_of_features_with_missing_values                  0.000000
percentage_of_categorical_columns_with_missing_values       0.000000
percentage_of_categorical_values_with_missing_values        0.000000
percentage_of_numeric_columns_with_missing_values           0.000000
percentage_of_numeric_values_with_missing_values            0.000000
number_of_numeric_features                                  7.000000
number_of_categorical_features                             13.000000
ratio_numerical_features          

Podemos personalizar los meta-features propios usando esta sintaxis:

In [40]:
from amltk.metalearning import MetaFeature

# crear una meta-features personalizada
class TotalValues(MetaFeature):

    @classmethod
    def compute(
        cls,
        x: pd.DataFrame, # features del dataset(filas,columnas)
        y: pd.Series | pd.DataFrame, # variable objetivo del dataset
        dependancy_values: dict,
    ) -> int:
        return int(x.shape[0] * x.shape[1])

mfs = compute_metafeatures(X, y, features=[TotalValues])
print("----------------------------------------------------")
print(f"La cantidad de metafeures calculadas es: {len(mfs)}")
print("----------------------------------------------------")
print(mfs)

----------------------------------------------------
La cantidad de metafeures calculadas es: 1
----------------------------------------------------
total_values    20000
dtype: int64


Puedes usar `DatasetStatistic`, lo que estás creando es una estadística intermedia del dataset, no una metacaracterística final. OpenML la calcula una sola vez por llamada a `compute_features()`.

Luego, cualquier metacaracterística (MetaFeature) que tenga esa estadística como dependencia puede usarla directamente desde el diccionario `dependancy_values`.

In [41]:
from amltk.metalearning import DatasetStatistic

class NAValues(DatasetStatistic):
    """A mask of all NA values in a dataset"""

    @classmethod
    def compute(
        cls,
        x: pd.DataFrame,
        y: pd.Series | pd.DataFrame,
        dependancy_values: dict,
    ) -> pd.DataFrame:
        return x.isna()
    

class PercentageNA(MetaFeature):
    """The percentage of values missing"""

    dependencies = (NAValues,)

    @classmethod
    def compute(
        cls,
        x: pd.DataFrame,
        y: pd.Series | pd.DataFrame,
        dependancy_values: dict,
    ) -> int:
        na_values = dependancy_values[NAValues]
        n_na = na_values.sum().sum()
        n_values = int(x.shape[0] * x.shape[1])
        return float(n_na / n_values)

mfs = compute_metafeatures(X, y, features=[PercentageNA])
print("----------------------------------------------------")
print(f"La cantidad de metafeures calculadas es: {len(mfs)}")
print("----------------------------------------------------")
mfs

----------------------------------------------------
La cantidad de metafeures calculadas es: 1
----------------------------------------------------


percentage_n_a    0.0
dtype: float64

Para ver la descripción de una metacaracterística específica, puedes llamar a .description() sobre ella.

In [42]:
from amltk.metalearning import metafeature_descriptions

descriptions = metafeature_descriptions()  # Obtiene un diccionario con todas las metacaracterísticas y sus descripciones
for name, description in descriptions.items():  # Itera sobre todas
    print("---")
    print(name)  # Imprime el nombre de la metacaracterística
    print("---")
    print(" * " + description)  # Imprime su descripción


---
instance_count
---
 * Number of instances in the dataset.
---
log_instance_count
---
 * Logarithm of the number of instances in the dataset.
---
number_of_classes
---
 * Number of classes in the dataset.
---
number_of_features
---
 * Number of features in the dataset.
---
log_number_of_features
---
 * Logarithm of the number of features in the dataset.
---
percentage_missing_values
---
 * Percentage of missing values in the dataset.
---
percentage_of_instances_with_missing_values
---
 * Percentage of instances with missing values.
---
percentage_of_features_with_missing_values
---
 * Percentage of features with missing values.
---
percentage_of_categorical_columns_with_missing_values
---
 * Percentage of categorical columns with missing values.
---
percentage_of_categorical_values_with_missing_values
---
 * Percentage of categorical values with missing values.
---
percentage_of_numeric_columns_with_missing_values
---
 * Percentage of numeric columns with missing values.
---
percent

# Dataset Distances

In [43]:
import pandas as pd
import openml

from amltk.metalearning import compute_metafeatures

def get_dataset(dataset_id: int) -> tuple[pd.DataFrame, pd.Series]:
    dataset = openml.datasets.get_dataset(
        dataset_id,
        download_data=True,
        download_features_meta_data=False,
        download_qualities=False,
    )
    X, y, _, _ = dataset.get_data(
        dataset_format="dataframe",
        target=dataset.default_target_attribute,
    )
    return X, y

d31 = get_dataset(31)
d3 = get_dataset(3)
d4 = get_dataset(4)

metafeatures_dict = {
    "dataset_31": compute_metafeatures(*d31),
    "dataset_3": compute_metafeatures(*d3),
    "dataset_4": compute_metafeatures(*d4),
}

metafeatures = pd.DataFrame(metafeatures_dict)
metafeatures

Could not download file from https://openml.org/datasets/0000/0031/dataset_31.pq: HTTPSConnectionPool(host='www.openml.org', port=443): Max retries exceeded with url: https://www.openml.org/datasets/0000/0031/dataset_31.pq (Caused by ResponseError('too many redirects'))
Failed to download parquet, fallback on ARFF.


,dataset_31,dataset_3,dataset_4
instance_count,1000.000000,3196.000000,57.000000
log_instance_count,6.907755,8.069655,4.043051
number_of_classes,2.000000,2.000000,2.000000
number_of_features,20.000000,36.000000,16.000000
log_number_of_features,2.995732,3.583519,2.772589
percentage_missing_values,0.000000,0.000000,0.357456
percentage_of_instances_with_missing_values,0.000000,0.000000,0.982456
percentage_of_features_with_missing_values,0.000000,0.000000,1.000000
percentage_of_categorical_columns_with_missing_values,0.000000,0.000000,1.000000
percentage_of_categorical_values_with_missing_values,0.000000,0.000000,0.410088


Ahora queremos conocer cuales `dataset_3` o `dataset_4` es similar a `dataset_31`:

In [44]:
from amltk.metalearning import dataset_distance

# extraer los meta-features del dataset31
target = metafeatures_dict.pop("dataset_31")

# en others ahora estan el resto, menos dataset31
others = metafeatures_dict

# calcula la distancia entre datasets, usado distancia euclidiana entre los vectores de metafeatures
distances = dataset_distance(target, others, distance_metric="l2")
print(distances)

dataset_4    19111.283139
dataset_3    95081.367356
Name: l2, dtype: float64


Es necesario escalar los meta-features porque los valores altos dominan la distancia Euclidiana, mientras que otras entre 0 y 1 tiene muy poco peso.

In [45]:
distances_scaled = dataset_distance(
    target,
    others,
    distance_metric="l2",
    scaler = "minmax" # aplicar min-max scaling sobre las filas
)

print(distances_scaled)

dataset_3    3.397475
dataset_4    3.624972
Name: l2, dtype: float64


Podemos ver que `dataset_3` es mas cercano a `dataset_31`. podemos usar tambien:

In [46]:
from sklearn.preprocessing import MinMaxScaler

distances = dataset_distance(
    target,
    others,
    distance_metric="l2",
    scaler=MinMaxScaler()
)
print(distances)

dataset_3    3.397475
dataset_4    3.624972
Name: l2, dtype: float64


# Portfolio Selection

Un portafolio en meta-learning es un conjunto de configuraciones que maximiza alguna nocion de cobertura sobre diferentes datasets o tareas.La intuición detrás de esto es que, de esta forma, cualquier nuevo dataset también quedará cubierto.

In [47]:
import pandas as pd

performances = {
    "c1": [90, 60, 20, 10],
    "c2": [20, 10, 90, 20],
    "c3": [10, 20, 40, 90],
    "c4": [90, 10, 10, 10],
}

portfolio = pd.DataFrame(performances, index=["dataset_1", "dataset_2", "dataset_3", "dataset_4"])
print(portfolio)


           c1  c2  c3  c4
dataset_1  90  20  10  90
dataset_2  60  10  20  10
dataset_3  20  90  40  10
dataset_4  10  20  90  10


Vemos aqui que tenemos 4 datasets y sus evaluaciones en 4 configuraciones. Ahora queremos `k=3` de esas configuraiones. La idea es seleccionar un subconjunto de estos algoritmos que maximice algún valor de utilidad del portafolio.

Esto se hace agregando una sola configuración del conjunto completo, una por una, hasta alcanzar k, comenzando desde un portafolio vacío.

In [48]:
from amltk.metalearning import portfolio_selection

selected_portfolio, trajectory = portfolio_selection(
    portfolio,
    k=3,
    scaler="minmax"
)

print(selected_portfolio)
print()
print(trajectory)

              c1     c3     c2
dataset_1  1.000  0.000  0.125
dataset_2  1.000  0.200  0.000
dataset_3  0.125  0.375  1.000
dataset_4  0.000  1.000  0.125

c1    0.53125
c3    0.84375
c2    1.00000
dtype: float64


La trayectoria nos indica qué configuración fue añadida en cada instante de tiempo, junto con la utilidad del portafolio después de añadir dicha configuración.

Sin embargo, no hemos especificado aún cómo se define exactamente la utilidad de un portafolio dado.

Podemos definir nuestra propia función para hacerlo:

In [49]:
def my_function(p: pd.DataFrame) -> float:
    # Take the maximum score for each dataset and then take the mean across them.
    return p.max(axis=1).mean()

selected_portfolio, trajectory = portfolio_selection(
    portfolio,
    k=3,
    scaler="minmax",
    portfolio_value=my_function,
)

print(selected_portfolio)
print()
print(trajectory)

              c1     c3     c2
dataset_1  1.000  0.000  0.125
dataset_2  1.000  0.200  0.000
dataset_3  0.125  0.375  1.000
dataset_4  0.000  1.000  0.125

c1    0.53125
c3    0.84375
c2    1.00000
dtype: float64


Esta noción de reducir (agregar) primero sobre todas las configuraciones para cada dataset y luego agregar esos resultados es lo suficientemente común como para que también podamos definir directamente estas operaciones, y el resto se realizará automáticamente.

In [50]:
import numpy as np

selected_portfolio, trajectory = portfolio_selection(
    portfolio,
    k=3,
    scaler="minmax",
    row_reducer=np.max,  # Reduce los valores por fila tomando el máximo (por dataset) (default)
    aggregator=np.mean,  # Agrega los valores de todos los datasets tomando la media (default)
)

print(selected_portfolio)
print()
print(trajectory)

              c1     c3     c2
dataset_1  1.000  0.000  0.125
dataset_2  1.000  0.200  0.000
dataset_3  0.125  0.375  1.000
dataset_4  0.000  1.000  0.125

c1    0.53125
c3    0.84375
c2    1.00000
dtype: float64
